In [2]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

# Dash 애플리케이션 초기화
app = dash.Dash(__name__)

# 애플리케이션 레이아웃 설정
app.layout = html.Div([
    dcc.Input(id='my-input', value='초기값', type='text'),
    html.Div(id='my-output') ])

# 콜백 정의
@app.callback(
    Output(component_id='my-output', component_property='children'),
    [Input(component_id='my-input', component_property='value')])
def update_output_div(input_value):
    return f'입력된 값: {input_value}'

if __name__ == '__main__':
    app.run_server(debug=True)  # 서버 실행


In [5]:
# 필요한 라이브러리 임포트
import dash
from dash import html, dcc, Input, Output
import plotly.express as px
import pandas as pd

# 외부 스타일시트 추가 (Bootstrap)
external_stylesheets = ['https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css',
                        'https://fonts.googleapis.com/css2?family=Noto+Sans+KR:wght@400;700&display=swap']# Noto Sans KR 폰트 추가
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

# 레이아웃 설정
app.layout = html.Div([
    html.H1("부동산 가격 변수 EDA", className='text-center', style={'font-family': 'Noto Sans KR'}),
    
    html.Div([
        html.Div([dcc.Graph(id='graph1')], className='col-md-6'),
        html.Div([dcc.Graph(id='graph2')], className='col-md-6'),
    ], className='row'),
    html.Div([
        html.Div([dcc.Graph(id='graph3')], className='col-md-6'),
        html.Div([dcc.Graph(id='graph4')], className='col-md-6'),
    ], className='row'),
    dcc.Markdown('''
        ## 프로젝트 개요
        이 대시보드는 **부동산 가격**의 다양한 영향 요소들을 분석하기 위한 것입니다. 
        분석된 데이터는 2015년부터 2024년까지의 기간을 포함하고 있으며, 다음과 같은 변수들을 포함합니다:

        - 매매지수
        - 금리
        - 대출 규모
        - 등등

        ### 분석 결과
        이 섹션에서는 각 변수들의 영향력과 상관관계를 **시각적으로 분석**합니다. 
        - '매매지수'는 부동산 가격 변동과 높은 상관관계를 보이며, 다른 변수들과의 관계도 중요합니다.
        
        #### 상세 분석
        - **매매지수**가 상승할 때, 대체적으로 부동산 가격도 상승하는 경향이 있습니다.
        - 금리의 변동은 부동산 시장에 직접적인 영향을 미치며, 특히 **단기 금리**의 영향이 큽니다.

        ###### 데이터 출처
        모든 데이터는 공공 데이터 소스에서 수집되었으며, 정확성을 기하기 위해 다양한 검증 과정을 거쳤습니다.
    ''', className='p-3', style={'font-family': 'Noto Sans KR'}),
    html.Button("Update Data", id='button', className='btn btn-primary mt-3')
])

# 콜백 설정
@app.callback(
    [Output('graph1', 'figure'),
     Output('graph2', 'figure'),
     Output('graph3', 'figure'),
     Output('graph4', 'figure')],
    [Input('button', 'n_clicks')]
)
def update_graph(n_clicks):
    df = pd.DataFrame({
        "Genre": ["Drama", "Comedy", "Action", "Thriller"],
        "Count": [1500, 1000, 500, 200]
    })
    fig1 = px.line(df, x='Genre', y=df['Count'].cumsum(), title="부동산 가격")
    fig2 = px.line(df, x='Genre', y='Count', title="금리")
    fig3 = px.scatter(df, x='Genre', y='Count', title="분포")
    fig4 = px.pie(df, names='Genre', values='Count', title="공사비 비중")
    return [fig1, fig2, fig3, fig4]

# 서버 실행
if __name__ == '__main__':
    app.run_server(debug=True)

In [2]:
# 필요한 라이브러리 임포트
import dash
from dash import html, dcc, Input, Output
import plotly.express as px
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import numpy as np
import plotly.figure_factory as ff




foundation_data = pd.read_csv('./JBC_DATA/0507_EDA.csv')
houseprice_columns = ['매매지수', '전세지수', '월세지수', 'KB주택매매가격지수', 'KB주택전세가격지수',
       '아파트매매실거래가격지수', '아파트매매가격지수', '아파트전세가격지수', '아파트월세통합가격지수',]

productindex_columns = ['소비자물가지수(총지수)', '소비자물가지수(주택임차료)',
       '소비자물가지수(전세)', '소비자물가지수(월세)', '생산자물가지수', '수입물가지수', '수출물가지수']

interate_columns=['한국은행 기준금리',
       '정부대출금리', '국민주택채권1종(5년)', '기대인플레이션']

# 나눠서 그릴 공간 생성
fig_LT = make_subplots(
    rows=3, cols=1,
    shared_xaxes=True,
    subplot_titles=("House prices","Commodity Index","Interest Rates"))

for column in houseprice_columns:
    if column == '아파트매매실거래가격지수':
        fig_LT.add_trace(go.Scatter(x=foundation_data['날짜'],y=foundation_data[column],
                              name=column,
                              mode='lines+markers'),
                 row=1, col=1)
    else:
        fig_LT.add_trace(go.Scatter(x=foundation_data['날짜'],y=foundation_data[column],
                              name=column),
                 row=1, col=1)

for column in productindex_columns:
    fig_LT.add_trace(go.Scatter(x=foundation_data['날짜'],y=foundation_data[column],
                              name=column),
                 row=2, col=1)

for column in interate_columns:
    fig_LT.add_trace(go.Scatter(x=foundation_data['날짜'],y=foundation_data[column],
                              name=column),
                 row=3, col=1)


one_df = foundation_data.drop(columns=['매매지수','전세지수','월세지수','KB주택매매가격지수',
                                       'KB주택전세가격지수','아파트매매가격지수','아파트전세가격지수',
                                       '아파트월세통합가격지수'])
one_df = one_df[['날짜', '아파트매매실거래가격지수','소비자물가지수(총지수)', '소비자물가지수(주택임차료)',
       '소비자물가지수(전세)', '소비자물가지수(월세)', '생산자물가지수', '수입물가지수', '수출물가지수']]


# '날짜' 열을 제외하고 상관관계 계산
correlation_matrix = one_df.drop(['날짜'], axis=1).corr()

# 상관관계 히트맵 생성
fig_hm = ff.create_annotated_heatmap(
    z=correlation_matrix.values,
    x=correlation_matrix.columns.tolist(),
    y=correlation_matrix.columns.tolist(),
    annotation_text=np.around(correlation_matrix.values, decimals=2),
    colorscale='Viridis',
    showscale=True
)
fig_hm.update_layout(title="변수 간 상관관계 히트맵", title_x=0.5, xaxis=dict(side='bottom'))


import dash
from dash import html, dcc, Input, Output
import plotly.express as px
import pandas as pd

# 데이터 로드
Commodity_data = pd.read_csv('국제상품가격.csv')
Commodity_data['날짜'] = pd.to_datetime(Commodity_data['날짜'])
commodity_columns = Commodity_data.columns[1:]
melted_data = Commodity_data.melt(id_vars=['날짜'], value_vars=commodity_columns, var_name='원자재', value_name='가격')

NationalMoney_data = pd.read_csv('본원통화.csv')
NationalMoney_data['날짜'] = pd.to_datetime(NationalMoney_data['날짜'])

Short_IR_data = pd.read_csv('국제금리.csv')
Short_IR_data['날짜'] = pd.to_datetime(Short_IR_data['날짜'])

Market_IR_data = pd.read_csv('시장금리.csv')
Market_IR_data.dropna(inplace=True)
Market_IR_data['날짜'] = pd.to_datetime(Market_IR_data['날짜'])

Producer_PI_data = pd.read_csv('생산자물가지수.csv')
Producer_PI_data.dropna(inplace=True)
Producer_PI_data['날짜'] = pd.to_datetime(Producer_PI_data['날짜'])

Consumer_PI_data = pd.read_csv('소비자물가지수.csv')
Consumer_PI_data.dropna(inplace=True)
Consumer_PI_data['날짜'] = pd.to_datetime(Consumer_PI_data['날짜'])

Bank_LoanIR_New_data = pd.read_csv('예금은행대출금리(신규).csv')
Bank_LoanIR_New_data.dropna(inplace=True)
Bank_LoanIR_New_data['날짜'] = pd.to_datetime(Bank_LoanIR_New_data['날짜'])

Bank_LoanIR_Balance_data = pd.read_csv('예금은행대출금리(잔액).csv')
Bank_LoanIR_Balance_data.dropna(inplace=True)
Bank_LoanIR_Balance_data['날짜'] = pd.to_datetime(Bank_LoanIR_Balance_data['날짜'])


# Commodity_data에서 원자재 이름을 추출 (첫 번째 열인 '날짜'는 제외)
commodity_columns = Commodity_data.columns[1:]

# 데이터를 재구조화하여 '날짜', '원자재', '가격' 형식의 DataFrame 생성
melted_data = Commodity_data.melt(id_vars=['날짜'], value_vars=commodity_columns, var_name='원자재', value_name='가격')

# 애니메이션 막대 그래프 생성
fig_am = px.bar(melted_data, 
             x='원자재', 
             y='가격', 
             color='원자재', 
             animation_frame='날짜',
             range_y=[melted_data['가격'].min(), melted_data['가격'].max()],
             log_y=True,
             title='날짜별 원자재 가격 변동')




# 외부 스타일시트 추가 (Bootstrap)
external_stylesheets = ['https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css',
                        'https://fonts.googleapis.com/css2?family=Noto+Sans+KR:wght@400;700&display=swap']# Noto Sans KR 폰트 추가
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

# 레이아웃 설정
app.layout = html.Div([
    html.H1("부동산 가격 변수 EDA", className='text-center', style={'font-family': 'Noto Sans KR'}),
    dcc.Dropdown(
        id='graph-dropdown',
        options=[
            {'label': '원자재 그래프', 'value': 'commodity'},
            {'label': '통화량 그래프', 'value': 'money'},
            {'label': '국제금리 그래프', 'value': 'interest_rate'},
            {'label': '시장금리', 'value': 'market_rate'},
            {'label': '생산자물가', 'value': 'producer_price'},
            {'label': '소비자물가', 'value': 'consumer_price'},
            {'label': '신규대출금리', 'value': 'new_loan_rate'},
            {'label': '잔액대출금리', 'value': 'balance_loan_rate'}
        ],
        value='commodity'
    ),
    dcc.Graph(id='graph-output'),
    dcc.Graph(id='heatmap-output'),
    dcc.Graph(id='example-graph-4',figure=fig_am),
    html.Div([
        html.Div([
            dcc.Graph(
                id='example-graph-2',figure=fig_LT)],className='col-md-6'),
        html.Div([
            dcc.Graph(
                id='example-graph-3',figure=fig_hm)],className='col-md-6'),
    ], className='row'),
    
    
    html.Button("Update Data", id='button', className='btn btn-primary mt-3')
])



# 콜백 정의
@app.callback(
    [Output('graph-output', 'figure'),
     Output('heatmap-output', 'figure')],
    [Input('graph-dropdown', 'value')]
)

def update_graph(graph_type):
    # 기본 temp 데이터 프레임 설정 (예시로 아파트 매매 실거래가격지수 포함)
    temp = one_df[['날짜', '아파트매매실거래가격지수']]
    temp['날짜'] = pd.to_datetime(temp['날짜'])

    if graph_type == 'commodity':
        fig = px.line(Commodity_data, x='날짜', y=Commodity_data.columns[1:], title='원자재 가격 변동')
        merged_df = pd.merge(temp, Commodity_data, on='날짜', how='inner')
    elif graph_type == 'money':
        fig = px.line(NationalMoney_data, x='날짜', y=NationalMoney_data.columns[1:], title='통화량 변동')
        merged_df = pd.merge(temp, NationalMoney_data, on='날짜', how='inner')
    elif graph_type == 'interest_rate':
        fig = px.line(Short_IR_data, x='날짜', y=Short_IR_data.columns[1:], range_y=(-1,12.5), title='국제금리 변동')
        merged_df = pd.merge(temp, Short_IR_data, on='날짜', how='inner')
    elif graph_type == 'market_rate':
        fig = px.line(Market_IR_data, x='날짜', y=Market_IR_data.columns[1:], title='시장금리')
        merged_df = pd.merge(temp, Market_IR_data, on='날짜', how='inner')
    elif graph_type == 'producer_price':
        fig = px.line(Producer_PI_data, x='날짜', y=Producer_PI_data.columns[1:], title='생산자물가지수')
        merged_df = pd.merge(temp, Producer_PI_data, on='날짜', how='inner')
    elif graph_type == 'consumer_price':
        fig = px.line(Consumer_PI_data, x='날짜', y=Consumer_PI_data.columns[1:], title='소비자물가지수')
        merged_df = pd.merge(temp, Consumer_PI_data, on='날짜', how='inner')
    elif graph_type == 'new_loan_rate':
        fig = px.line(Bank_LoanIR_New_data, x='날짜', y=Bank_LoanIR_New_data.columns[1:], title='예금은행대출금리(신규)')
        merged_df = pd.merge(temp, Bank_LoanIR_New_data, on='날짜', how='inner')
    else:
        fig = px.line(Bank_LoanIR_Balance_data, x='날짜', y=Bank_LoanIR_Balance_data.columns[1:], title='예금은행대출금리(잔액)')
        merged_df = pd.merge(temp, Bank_LoanIR_Balance_data, on='날짜', how='inner')

    # 상관관계 히트맵 생성
    correlation_matrix = merged_df.drop(['날짜'], axis=1).corr()
    heatmap = ff.create_annotated_heatmap(
        z=correlation_matrix.values,
        x=correlation_matrix.columns.tolist(),
        y=correlation_matrix.columns.tolist(),
        annotation_text=np.around(correlation_matrix.values, decimals=2),
        colorscale='Viridis',
        showscale=True
    )
    heatmap.update_layout(title=f"{graph_type} 데이터와 아파트 매매 가격의 상관관계", title_x=0.5, xaxis=dict(side='bottom'))

    return fig, heatmap

# 서버 실행
if __name__ == '__main__':
    app.run_server(debug=True)


FileNotFoundError: [Errno 2] No such file or directory: './JBC_DATA/0507_EDA.csv'